[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/Pesquisa-Opercional-III-A/blob/main/8_CPM_Impacto.ipynb)

# Gestão de Impacto em Projetos

In [ ]:
# MODELO
M = pe.ConcreteModel() # Criando o modelo concreto para o nosso problema

#Atividades
A = M.atividades = pe.Set(initialize= ["A", "B", "C", "D", "E", "F", "G", "H"])
A_ = M.atividades_sem_predecessores = pe.Set(initialize= ["C", "D", "E", "F", "G", "H", "T"])

# Parâmetros
M.c_impacto = pe.Param(M.atividades, initialize={"A": 6, "B": 12, "C": 4, "D": 7, "E": 10, "F": 9, "G": 6, "H": 8})
M.t_normal = pe.Param(M.atividades, initialize={"A": 5, "B": 3, "C": 4, "D": 6, "E": 5, "F": 7, "G": 9, "H": 8})
M.t_impacto = pe.Param(M.atividades, initialize={"A": 2, "B": 1, "C": 2, "D": 3, "E": 1, "F": 3, "G": 4, "H": 2}) 

# VARIÁVEIS
M.x = pe.Var(A, domain= pe.PositiveReals) # Definindo a variável x
M.y = pe.Var(A_, domain= pe.PositiveReals) # Definindo a variável y

# FUNÇÃO OBJETIVO
z = M.z = pe.Objective(rule = lambda M: sum(M.x[i] * M.c_impacto[i] for i in M.atividades), sense = pe.minimize) # Definindo a função objetivo

# RESTRIÇÃO
M.R1 = pe.Constraint(A, expr= lambda M, i: M.x[i] <= M.t_impacto[i]) # Definindo uma restrição de igualdade

M.R_CA = pe.Constraint(expr= M.y["C"] >= 0 + M.t_normal["A"] - M.x["A"])
M.R_DA = pe.Constraint(expr= M.y["D"] >= 0 + M.t_normal["A"] - M.x["A"])
M.R_EB = pe.Constraint(expr= M.y["E"] >= 0 + M.t_normal["B"] - M.x["B"])
M.R_FB = pe.Constraint(expr= M.y["F"] >= 0 + M.t_normal["B"] - M.x["B"])

M.R_GC = pe.Constraint(expr= M.y["G"] >= M.y["C"] + M.t_normal["C"] - M.x["C"])
M.R_GE = pe.Constraint(expr= M.y["G"] >= M.y["E"] + M.t_normal["E"] - M.x["E"])


M.R_HD = pe.Constraint(expr= M.y["H"] >= M.y["D"] + M.t_normal["D"] - M.x["D"])
M.R_HF = pe.Constraint(expr= M.y["H"] >= M.y["F"] + M.t_normal["F"] - M.x["F"])

M.R_TG = pe.Constraint(expr= M.y["T"] >= M.y["G"] + M.t_normal["G"] - M.x["G"])
M.R_TH = pe.Constraint(expr= M.y["T"] >= M.y["H"] + M.t_normal["H"] - M.x["H"])

M.Termino = pe.Constraint(expr = M.y["T"]<=16)

# SOLUCIONANDO O MODELO
pe.SolverFactory('glpk').solve(M)

# Imprime o resultado
print(f"z= {pe.value(M.z)}")
for k in M.x.keys(): 
    if pe.value(M.x[k]) > 0: 
        print (f"x_{k} = {pe.value(M.x[k])}")

z= 28.0
x_A = 1.0
x_G = 1.0
x_H = 2.0
